In [23]:

import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, Input,Layer
from scipy.fftpack import dct
# Configurazione per la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limita la memoria GPU allocata (opzionale)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Permette allocazione dinamica della memoria
        print(f"{len(gpus)} GPU(s) disponibile/i.")
    except RuntimeError as e:
        print(e)
else:
    print("Nessuna GPU disponibile, verrà usata la CPU.")

# Carica il dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

def apply_dct(data):
    # Esegui la DCT bidimensionale sull'immagine
    # La `type=2` rappresenta la DCT-II, che è la più comune
    return dct(dct(data, axis=0, norm='ortho'), axis=1, norm='ortho')

X_train_dct = np.array([apply_dct(img) for img in X_train])  # Applica la DCT alle immagini di training
X_test_dct = np.array([apply_dct(img) for img in X_test])  # Applica la DCT alle immagini di test


print("Forma di X_train_dct:", X_train_dct.shape)
print("Forma di X_test_dct:", X_test_dct.shape)

1 GPU(s) disponibile/i.
Forma di X_train_dct: (60000, 28, 28)
Forma di X_test_dct: (10000, 28, 28)


In [24]:
filtri = np.random.rand(32, 3, 3)

filtri_dct = np.array([apply_dct(img) for img in filtri])  # Applica la DCT alle immagini di training


# Stampa i filtri generati
for i, filtro in enumerate(filtri_dct):
    print(f"Filtro {i+1}:\n{filtro}\n")

Filtro 1:
[[ 0.97722438 -0.48549028  0.06127682]
 [-0.03606768  0.0917343  -0.1389121 ]
 [-0.26961112  0.36874774  0.11591712]]

Filtro 2:
[[ 2.11248623 -0.18717925 -0.08115528]
 [-0.00837894  0.05051863 -0.00576   ]
 [ 0.42908952  0.35642914  0.19320933]]

Filtro 3:
[[ 1.06669115  0.04722854  0.03681574]
 [-0.18604402 -0.75636404  0.06034862]
 [ 0.13166751  0.29198734  0.27602571]]

Filtro 4:
[[ 1.72371127  0.34437157 -0.10014884]
 [ 0.19564837 -0.35175085  0.53414074]
 [ 0.06143399 -0.51983655  0.04448111]]

Filtro 5:
[[ 1.20903258  0.03421138  0.39557056]
 [-0.16599748  0.30709143 -0.16323028]
 [-0.21476723 -0.09222288  0.22292154]]

Filtro 6:
[[ 1.46075721 -0.19133598  0.06356943]
 [-0.03234115  0.06230536  0.15576205]
 [ 0.21231456 -0.09337227 -0.32452842]]

Filtro 7:
[[ 1.31322708 -0.11489648  0.30948551]
 [-0.22073133 -0.23893432  0.19523494]
 [-0.36031242  0.09200808 -0.37581205]]

Filtro 8:
[[ 1.69278937  0.22826007  0.38077898]
 [-0.02672125 -0.4003427  -0.53594029]
 [-0.1556

In [25]:
#rimuovi tutte le colonne apparte prime 3 in tutte le immagini di training e test
X_train_dct = X_train_dct[:, :, :3]
X_test_dct = X_test_dct[:, :, :3]



#printa le shape

print("Forma di X_train_dct:", X_train_dct.shape)
print("Forma di X_test_dct:", X_test_dct.shape)


Forma di X_train_dct: (60000, 28, 3)
Forma di X_test_dct: (10000, 28, 3)


In [26]:
class IDCTLayer(Layer):
    def __init__(self, **kwargs):
        super(IDCTLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # IDCT lungo l'asse delle righe e poi delle colonne
        def idct_2d(tensor):
            # Applica la IDCT lungo le righe (asse=-1)
            x = tf.signal.idct(tensor, type=2, norm='ortho', axis=-1)
            # Trasponi per applicare la IDCT lungo le colonne
            x = tf.transpose(x, perm=[0, 1, 3, 2])  # Scambia -2 (righe) con -1 (colonne)
            x = tf.signal.idct(x, type=2, norm='ortho', axis=-1)
            # Ritorna alla disposizione originale
            x = tf.transpose(x, perm=[0, 1, 3, 2])  # Ritorna alla forma originale
            return x

        # Applica la IDCT batch-wise e canale per canale
        return idct_2d(inputs)

In [27]:
model = Sequential([
    Input(shape=(28, 3)),  
    Flatten(),             
    Dense(228, activation='relu'),
    Dense(576, activation='relu'),
    Dense(1152, activation='relu'),
    Reshape((3, 3, 128)),  
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [28]:
# Compila il modello
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Addestra il modello
history = model.fit(X_train_dct, y_train, epochs=30, batch_size=64, validation_split=0.2)

# Valuta il modello
test_loss, test_acc = model.evaluate(X_test_dct, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")


Epoch 1/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6649 - loss: 8.3668 - val_accuracy: 0.8033 - val_loss: 0.5690
Epoch 2/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8106 - loss: 0.5245 - val_accuracy: 0.8234 - val_loss: 0.4867
Epoch 3/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8310 - loss: 0.4577 - val_accuracy: 0.8308 - val_loss: 0.4723
Epoch 4/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8433 - loss: 0.4293 - val_accuracy: 0.8344 - val_loss: 0.4618
Epoch 5/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8527 - loss: 0.4038 - val_accuracy: 0.8317 - val_loss: 0.4690
Epoch 6/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8594 - loss: 0.3883 - val_accuracy: 0.8418 - val_loss: 0.4579
Epoch 7/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8645 - loss: 0.3716 - val_accuracy: 0.8376 - val_loss: 0.4700
Epoch 8/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8683 - loss: 0.3591 - val_accuracy: 0.

In [29]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape

# Configurazione per la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limita la memoria GPU allocata (opzionale)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Permette allocazione dinamica della memoria
        print(f"{len(gpus)} GPU(s) disponibile/i.")
    except RuntimeError as e:
        print(e)
else:
    print("Nessuna GPU disponibile, verrà usata la CPU.")

# Carica il dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()



# Crea il modello
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])


# Compila il modello
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Addestra il modello
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Valuta il modello
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")


1 GPU(s) disponibile/i.
Epoch 1/10
319/750 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7094 - loss: 10.2375

KeyboardInterrupt: 